In [99]:
import numpy as np
import scipy as sp
import pandas as pd
import re

In [12]:
logdir='/Users/ian/Library/Group Containers/G69SCX94XU.duck/Library/Application Support/duck/Volumes/hawk/propagation'

In [176]:
logfiles=['propagation-938198.hawk-pbs5-20201201T0458.out',
          'propagation-938199.hawk-pbs5-20201201T0517.out',
          'propagation-938200.hawk-pbs5-20201201T0531.out',
          'propagation-938201.hawk-pbs5-20201201T0533.out',
          'propagation-938202.hawk-pbs5-20201201T0535.out',
          'propagation-938203.hawk-pbs5-20201201T0248.out',
          'propagation-938204.hawk-pbs5-20201201T0250.out',
          'propagation-938205.hawk-pbs5-20201201T0308.out',
          'propagation-938206.hawk-pbs5-20201201T0315.out',
          'propagation-938207.hawk-pbs5-20201201T0335.out',
          'propagation-938213.hawk-pbs5-20201201T0316.out',
          'propagation-938214.hawk-pbs5-20201201T0319.out',
          'propagation-938215.hawk-pbs5-20201201T0320.out',
          'propagation-938216.hawk-pbs5-20201201T0327.out',
          'propagation-938217.hawk-pbs5-20201201T0327.out',
          'propagation-938218.hawk-pbs5-20201201T0149.out',
          'propagation-938219.hawk-pbs5-20201201T0210.out',
          'propagation-938220.hawk-pbs5-20201201T0224.out',
          'propagation-938221.hawk-pbs5-20201201T0227.out',
          'propagation-938222.hawk-pbs5-20201201T0228.out',
          
          'propagation-938224.hawk-pbs5-20201201T1230.out',
          'propagation-938225.hawk-pbs5-20201201T1342.out',
          'propagation-938226.hawk-pbs5-20201201T1354.out',
          'propagation-938885.hawk-pbs5-20201201T1335.out',
          'propagation-938887.hawk-pbs5-20201201T1335.out',
          'propagation-938888.hawk-pbs5-20201201T1336.out',
          'propagation-938889.hawk-pbs5-20201201T1357.out',
          'propagation-938890.hawk-pbs5-20201201T1358.out',
          'propagation-938891.hawk-pbs5-20201201T1335.out',
          'propagation-938892.hawk-pbs5-20201201T1335.out',
          'propagation-938893.hawk-pbs5-20201201T1336.out',
          'propagation-938894.hawk-pbs5-20201201T1337.out',
          'propagation-938895.hawk-pbs5-20201201T1339.out',
          'propagation-938896.hawk-pbs5-20201201T1338.out',
          'propagation-938897.hawk-pbs5-20201201T1342.out',
          'propagation-938898.hawk-pbs5-20201201T1343.out',
          'propagation-938899.hawk-pbs5-20201201T1344.out',
          'propagation-938900.hawk-pbs5-20201201T1345.out',
          'propagation-939320.hawk-pbs5-20201201T1816.out',
          'propagation-939321.hawk-pbs5-20201201T1831.out',
          'propagation-939322.hawk-pbs5-20201201T2232.out'
          
         ]

In [13]:
file=f'{logdir}/{logfiles[0]}'

In [174]:
def parse_file(logdir, filename):
    ['jobid', 'topic', 'nodes', 'cores', 'runtime', 'mae_mean_retweets', 'mae_retweet_probability', 'sources',
     'samples', 'mean_retweets']
    filename = logdir + '/' + filename
    with open(filename, 'r') as file:
        res = {
            'jobid': re.search('-(\d+\.hawk-pbs5)', filename).group(1),
            'nodes': int(file.readline()),
            'cores': int(file.readline()),
        }
        for l in file:
            res.update(parse_stats(l))
        resultfile = f'{logdir}/results-{res["topic"]}-{res["jobid"]}.csv'
        results = pd.read_csv(resultfile)
        res['mean_retweets'] = results.simulation_mean_retweets.mean()
        return res
    
def parse_stats(s):
    fields=(
            ('topic', 'topic: (.+)$', str),
            ('mae_mean_retweets', 'MAE: retweet_probability: .+ mean_retweets: (.+)$', float),
            ('mae_retweet_probability', 'MAE: retweet_probability: (.+) mean_retweets: .+$', float),
            ('features', '(\d+) features, \d+ sources, \d+ samples$', int),
            ('sources', '\d+ features, (\d+) sources, \d+ samples$', int),
            ('samples', '\d+ features, \d+ sources, (\d+) samples$', int),
            ('runtime', 'system (.+)elapsed', str)
            )
    res={}
    for f in fields:
        m = re.search(f[1], s)
        if m:
            res[f[0]]=f[2](m.group(1))
    return res

from datetime import datetime
def parse_time(time):
    try:
        t=datetime.strptime(time, '%M:%S.%f')
    except:
        t=datetime.strptime(time, '%H:%M:%S')
    finally:
        return t.strftime('%H:%M:%S.%f')
    
def time_to_seconds(x):
    from datetime import timedelta
    return timedelta(hours=x.hour, minutes=x.minute, seconds=x.second, microseconds=x.microsecond).total_seconds()
    

def parse_files(logdir, logfiles):
    r=pd.DataFrame([parse(logdir, f) for f in logfiles])
    r.set_index('jobid',inplace=True)
    r['runtime']=r.runtime.apply(parse_time)
    r['runtime_s']=r.runtime.apply(lambda x: time_to_seconds(datetime.strptime(x, '%H:%M:%S.%f')))
    r['total_tweets'] = r.features*r.sources*r.samples
    r['total_retweets'] = r.features*r.sources*r.samples*r.mean_retweets
    return r

In [160]:
from datetime import timedelta
timedelta(hours=1,seconds=2,milliseconds=100).strftime('%H:%M:%S.%f')

AttributeError: 'datetime.timedelta' object has no attribute 'strftime'

In [177]:
r=parse_files(logdir,logfiles)
r.to_csv('runs.csv')
r

,nodes,cores,topic,features,sources,samples,mae_mean_retweets,mae_retweet_probability,runtime,mean_retweets,runtime_s,total_tweets,total_retweets
jobid,,,,,,,,,,,,,
938198.hawk-pbs5,64,8192,bvb_20200409,121,1000,8000,11.929145,0.000111,00:19:04.890000,13.780517,1144.89,968000000,1.333954e+10
938199.hawk-pbs5,64,8192,fpoe_20200311,132,1000,8000,8.815658,0.000136,00:13:48.560000,14.327051,828.56,1056000000,1.512937e+10
938200.hawk-pbs5,64,8192,neos_20200311,91,1000,8000,2.544195,0.000127,00:08:46.000000,6.738203,526.00,728000000,4.905411e+09
938201.hawk-pbs5,64,8192,schalke_20200409,124,1000,8000,10.344461,0.000102,00:19:31.580000,11.618538,1171.58,992000000,1.152559e+10
938202.hawk-pbs5,64,8192,vegan_20200407,81,1000,8000,1.181722,0.000102,00:12:03.130000,2.218609,723.13,648000000,1.437659e+09
938203.hawk-pbs5,32,4096,bvb_20200409,121,1000,8000,11.823619,0.000107,00:20:02.720000,13.701808,1202.72,968000000,1.326335e+10
938204.hawk-pbs5,32,4096,fpoe_20200311,132,1000,8000,8.760656,0.000126,00:14:41.230000,14.279291,881.23,1056000000,1.507893e+10
938205.hawk-pbs5,32,4096,neos_20200311,91,1000,8000,2.511963,0.000122,00:06:52.250000,6.811728,412.25,728000000,4.958938e+09
938206.hawk-pbs5,32,4096,schalke_20200409,124,1000,8000,9.963786,0.000109,00:19:55.330000,11.212659,1195.33,992000000,1.112296e+10


In [112]:
r

,jobid,nodes,cores,topic,sources,samples,mae_mean_retweets,mae_retweet_probability,runtime,mean_retweets
0,938198.hawk-pbs5,64,8192,bvb_20200409,1000,8000,11.929145,0.000111,19:04.89,13.780517
1,938199.hawk-pbs5,64,8192,fpoe_20200311,1000,8000,8.815658,0.000136,13:48.56,14.327051
2,938200.hawk-pbs5,64,8192,neos_20200311,1000,8000,2.544195,0.000127,8:46.00,6.738203
3,938201.hawk-pbs5,64,8192,schalke_20200409,1000,8000,10.344461,0.000102,19:31.58,11.618538
4,938202.hawk-pbs5,64,8192,vegan_20200407,1000,8000,1.181722,0.000102,12:03.13,2.218609
5,938203.hawk-pbs5,32,4096,bvb_20200409,1000,8000,11.823619,0.000107,20:02.72,13.701808
6,938204.hawk-pbs5,32,4096,fpoe_20200311,1000,8000,8.760656,0.000126,14:41.23,14.279291
7,938205.hawk-pbs5,32,4096,neos_20200311,1000,8000,2.511963,0.000122,6:52.25,6.811728
8,938206.hawk-pbs5,32,4096,schalke_20200409,1000,8000,9.963786,0.000109,19:55.33,11.212659
9,938207.hawk-pbs5,32,4096,vegan_20200407,1000,8000,1.132076,0.000100,6:44.76,2.160293


In [84]:
p=r.pivot(index='nodes',columns='topic',values='runtime')
p.to_csv('scaling.csv')

In [85]:
p=r.pivot(index='nodes',columns='topic',values='mae_mean_retweets')
p.to_csv('mae.csv')

In [92]:
r

,jobid,nodes,cores,topic,sources,samples,mae_mean_retweets,mae_retweet_probability,runtime,mean_retweets
0,938198.hawk-pbs5,64,8192,bvb_20200409,1000,8000,11.929145,0.000111,0:19:04.89,13.780517
1,938199.hawk-pbs5,64,8192,fpoe_20200311,1000,8000,8.815658,0.000136,0:13:48.56,14.327051
2,938200.hawk-pbs5,64,8192,neos_20200311,1000,8000,2.544195,0.000127,0:8:46.00,6.738203
3,938201.hawk-pbs5,64,8192,schalke_20200409,1000,8000,10.344461,0.000102,0:19:31.58,11.618538
4,938202.hawk-pbs5,64,8192,vegan_20200407,1000,8000,1.181722,0.000102,0:12:03.13,2.218609
5,938203.hawk-pbs5,32,4096,bvb_20200409,1000,8000,11.823619,0.000107,0:20:02.72,13.701808
6,938204.hawk-pbs5,32,4096,fpoe_20200311,1000,8000,8.760656,0.000126,0:14:41.23,14.279291
7,938205.hawk-pbs5,32,4096,neos_20200311,1000,8000,2.511963,0.000122,0:6:52.25,6.811728
8,938206.hawk-pbs5,32,4096,schalke_20200409,1000,8000,9.963786,0.000109,0:19:55.33,11.212659
9,938207.hawk-pbs5,32,4096,vegan_20200407,1000,8000,1.132076,0.000100,0:6:44.76,2.160293
